In [1]:
# Import PyTorch library for tensor operations and deep learning
import torch

# Import torchvision's datasets and image transformations
from torchvision import datasets, transforms

# Import DataLoader for batch loading, TensorDataset for wrapping data
from torch.utils.data import DataLoader, TensorDataset

# Import neural network modules and optimizers from PyTorch
from torch import nn, optim

# Import PIL for image loading and manipulation
from PIL import Image

# Import NumPy for numerical operations (arrays, math, etc.)
import numpy as np

# Import os for interacting with the operating system (file paths, etc.)
import os

# Import matplotlib for plotting graphs and images
import matplotlib.pyplot as plt

# --- 1. Preprocessing and Transforms ---

# Define the preprocessing pipeline for the training data:
transform_train = transforms.Compose([

    transforms.ToTensor(),                     # Convert images to PyTorch tensors
    transforms.Normalize((0.5,), (0.5,))        # Normalize tensor values to have mean 0.5 and std deviation 0.5
])

# Define the preprocessing pipeline for the testing/validation data:
transform_test = transforms.Compose([
    transforms.ToTensor(),                     # Convert images to PyTorch tensors (no random changes)
    transforms.Normalize((0.5,), (0.5,))        # Normalize tensor values the same way as training data
])


In [2]:
# Define a custom function to load images and labels from a folder
def ProjectDataLoader_MNIST(train=True, transform=None):
    mnist_dataset = datasets.MNIST(
        root='./MNIST_data',
        train=train,
        download=True,
        transform=transform if transform else transforms.ToTensor()
    )

    images = []
    labels = []

    for img, label in mnist_dataset:
        images.append(img)
        labels.append(label)

    return torch.stack(images), torch.tensor(labels).long()


# Just pass the transform and let PyTorch handle batching efficiently
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

transform_train = transforms.Compose([
    transforms.RandomRotation(15),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.05)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

trainset = datasets.MNIST(
    root='./MNIST_data',
    train=True,
    download=True,
    transform=transform_train
)

trainloader = DataLoader(trainset, batch_size=16, shuffle=True)



In [3]:
model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(28*28, 512), nn.ReLU(),
    nn.Linear(512, 256), nn.ReLU(),
    nn.Linear(256, 64), nn.ReLU(),
    nn.Linear(64, 10)
)


In [4]:
import torch
import matplotlib.pyplot as plt

# Tracking
train_loss_history = []
val_acc_history = []

# Loss function and optimizer (adjust as needed)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

epochs = 10  # Change as needed
for epoch in range(epochs):
    model.train()
    running_loss = 0

    for images, labels in trainloader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    avg_loss = running_loss / len(trainloader)
    train_loss_history.append(avg_loss)

    # --- Validation Step ---
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in testloader:
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_acc = 100 * correct / total
    val_acc_history.append(val_acc)

    print(f"Epoch {epoch+1}/{epochs} - Train Loss: {avg_loss:.4f} - Val Accuracy: {val_acc:.2f}%")


NameError: name 'testloader' is not defined

In [ ]:
# --- 7. Testing on MNIST Test Set ---

# Set the model to evaluation mode (disables dropout, etc.)
model.eval()

# Initialize counters for correct predictions and total samples
correct = 0
total = 0

# Turn off gradient calculations (makes testing faster and uses less memory)
with torch.no_grad():
    for images, labels in trainloader:         # Loop through the test data
        outputs = model(images)                # Forward pass: get model predictions
        _, predicted = torch.max(outputs, 1)   # Take the class with the highest score
        total += labels.size(0)                # Add the number of samples in this batch
        correct += (predicted == labels).sum().item()  # Count how many predictions were correct

# Calculate and print the final test set accuracy as a percentage
print(f"\nAccuracy on MNIST test set: {100 * correct / total:.2f}%")


In [ ]:
# Import matplotlib for plotting
import matplotlib.pyplot as plt

# Create a new figure with a specific size
plt.figure(figsize=(8, 5))

# Plot training loss over epochs with markers on each point
plt.plot(range(1, len(loss_history)+1), loss_history, marker='o')

# Set plot title and labels
plt.title('Training Loss Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')

# Add a grid to the plot for better readability
plt.grid(True)

# Display the plot
plt.show()

# --- 8. (Optional) Visualize a Few Test Predictions ---

# Define a function to show predictions from the test set
def show_mnist_predictions(model, trainloader):
    model.eval()                         # Set the model to evaluation mode
    images, labels = next(iter(trainloader))  # Get the first batch of test images and labels
    outputs = model(images)               # Get model predictions for this batch
    _, preds = torch.max(outputs, 1)      # Get predicted class labels

    # Create a 2x5 grid of subplots to display 10 images
    fig, axs = plt.subplots(2, 5, figsize=(12, 5))
    for i, ax in enumerate(axs.flat):     # Loop through each subplot
        ax.imshow(images[i][0], cmap='gray')  # Show the image (single channel grayscale)
        ax.set_title(f"Pred: {preds[i].item()} | Label: {labels[i].item()}")  # Show predicted and true label
        ax.axis('off')                    # Hide the axes for clarity
    plt.tight_layout()                    # Adjust spacing to prevent overlap
    plt.show()                            # Display the figure

# Call the function to show a few test set predictions
show_mnist_predictions(model, trainloader)


In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(range(1, epochs + 1), val_acc_history, marker='o', label='Validation Accuracy')
plt.title('Validation Accuracy Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

plt.figure(figsize=(8, 5))
plt.plot(range(1, epochs + 1), train_loss_history, marker='x', label='Training Loss')
plt.title('Training Loss Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()
